In [ ]:
'''!pip uninstall -y torch torchvision transformers tf-keras huggingface_hub accelerate tqdm ipywidgets
!pip cache purge
!df -h'''

'!pip uninstall -y torch torchvision transformers tf-keras huggingface_hub accelerate tqdm ipywidgets\n!pip cache purge\n!df -h'

In [ ]:
'''!pip install torch
!pip install torchvision
!pip install transformers
!pip install tf-keras
!pip install huggingface_hub
!pip install accelerate
!pip install ipywidgets
!pip install tqdm'''

'!pip install torch\n!pip install torchvision\n!pip install transformers\n!pip install tf-keras\n!pip install huggingface_hub\n!pip install accelerate\n!pip install ipywidgets\n!pip install tqdm'

In [ ]:
import torch
from transformers import pipeline
from huggingface_hub import login

E0000 00:00:1755531619.109160    1503 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755531619.117514    1503 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755531619.137037    1503 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755531619.137063    1503 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755531619.137065    1503 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755531619.137067    1503 computation_placer.cc:177] computation placer already registered. Please check linka

In [ ]:
# Access Token from Hugging Face
login(token = "use_your_own")

In [ ]:
# Setting pipeline for ICL
device = "cuda"

model_id = "meta-llama/Llama-3.2-1B"

pipe = pipeline(
    "text-generation",
    model = model_id,
    torch_dtype = torch.bfloat16,
    device_map = device
)

tokenizer = pipe.tokenizer

Device set to use cuda


# 1st Scenario: Providing Advice

In [ ]:
prompt ='''Based on the patient's medical parameters and heart disease diagnosis, provide appropriate medical advice:

Patient info: The patient is a 55 years old Female.
BMI is 34.93.
The patient does not smoke, does not drink alcohol, and is physically active.
Blood pressure is 140 / 90.
Blood pressure category: Hypertension Stage 2.
Cholesterol is well above normal.
Glucose is normal. Does the patient have heart disease? Answer: Yes

Advice: Your blood pressure and cholesterol levels are significantly elevated, which increases your cardiovascular risk.
It is important to work closely with your healthcare provider to manage these factors through lifestyle changes and possibly medication.
Regular monitoring and follow-up visits are recommended to reduce the risk of complications.


Patient info: The patient is a 50 years old Male.
BMI is 21.97.
The patient does not smoke, does not drink alcohol, and is physically active.
Blood pressure is 110 / 80.
Blood pressure category: Hypertension Stage 1.
Cholesterol is normal.
Glucose is normal. Does the patient have heart disease? Answer: No

Advice: Your vital signs and lab results are within normal ranges, which is reassuring.
Continue maintaining your healthy lifestyle, including regular physical activity and a balanced diet, to keep your heart in good condition.
Regular check-ups are still important to monitor any future changes.


Patient info: The patient is a 45 years old Male.
BMI is 37.86.
The patient does not smoke, does not drink alcohol, and is physically not active.
Blood pressure is 120 / 80.
Blood pressure category: Hypertension Stage 1.
Cholesterol is normal.
Glucose is normal. Does the patient have heart disease? Answer: Yes

Advice:'''

In [ ]:
output = pipe(prompt,
              max_new_tokens = 200,
              #temperature = 0.5, # controls randomness for more varied outputs
              #top_p = 0.95, # controls how much prob mass is considered (top 95%)
              repetition_penalty = 1.2, # penalises repeating tokens
              do_sample = False,
              return_full_text = False, # prompt included
              #early_stopping = True, # stops generating when EOS token is reached
             pad_token_id = tokenizer.eos_token_id)

print(output[0]["generated_text"])

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 You may be at increased risk for developing coronary artery disease (CAD) due to high LDL-C level.
Your doctor will likely recommend medications or lifestyle modifications such as weight loss and exercise to help lower your LDL-C levels.




# 2nd Scenario: Express Uncertainty/Confidence

In [ ]:
prompt2 ='''Based on the patient's medical parameters and heart disease diagnosis, provide appropriate medical advice:

Patient info: The patient is a 55 years old Female.
BMI is 34.93.
The patient does not smoke, does not drink alcohol, and is physically active.
Blood pressure is 140 / 90.
Blood pressure category: Hypertension Stage 2.
Cholesterol is well above normal.
Glucose is normal. Does the patient have heart disease? Answer: Yes

Advice: Your blood pressure and cholesterol levels appear to be significantly elevated, which likely increases your cardiovascular risk.
It would be prudent to work closely with your healthcare provider to manage these factors, possibly through lifestyle changes and medication.
Regular monitoring and follow-up visits are strongly recommended to help reduce the risk of complications.
While these recommendations are based on current data, individual responses can vary, so ongoing evaluation is important.


Patient info: The patient is a 50 years old Male.
BMI is 21.97.
The patient does not smoke, does not drink alcohol, and is physically active.
Blood pressure is 110 / 80.
Blood pressure category: Hypertension Stage 1.
Cholesterol is normal.
Glucose is normal. Does the patient have heart disease? Answer: No

Advice: Your vital signs and lab results are generally within normal ranges, which is reassuring.
Maintaining your healthy lifestyle, including regular physical activity and a balanced diet, will help keep your heart in good condition.
However, it remains important to have regular check-ups to monitor any potential future changes.
Although your current risk appears low, health status can evolve over time.


Patient info: The patient is a 45 years old Male.
BMI is 37.86.
The patient does not smoke, does not drink alcohol, and is physically not active.
Blood pressure is 120 / 80.
Blood pressure category: Hypertension Stage 1.
Cholesterol is normal.
Glucose is normal. Does the patient have heart disease? Answer: Yes

Advice:'''

In [ ]:
output2 = pipe(prompt2,
              max_new_tokens = 200,
              #temperature = 0.5, # controls randomness for more varied outputs
              #top_p = 0.95, # controls how much prob mass is considered (top 95%)
              repetition_penalty = 1.2, # penalises repeating tokens
              do_sample = False,
              return_full_text = False, # prompt included
              #early_stopping = True, # stops generating when EOS token is reached
             pad_token_id = tokenizer.eos_token_id)

print(output2[0]["generated_text"])

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 Based on your age and other clinical information provided,
your hypertension may require treatment or management.

Your doctor should discuss options for managing this issue during an appointment.
In addition to medications, you might also benefit from making some simple lifestyle adjustments such as eating healthier foods, exercising regularly, reducing stress, etc., that could improve overall wellness.






# 3rd Scenario: Follow-up Questions

In [ ]:
prompt3 ='''Based on the patient's medical parameters and heart disease diagnosis, provide appropriate medical advice:

Patient info: The patient is a 55 years old Female.
BMI is 34.93.
The patient does not smoke, does not drink alcohol, and is physically active.
Blood pressure is 140 / 90.
Blood pressure category: Hypertension Stage 2.
Cholesterol is well above normal.
Glucose is normal. Does the patient have heart disease? Answer: Yes

Advice: Your blood pressure and cholesterol levels are significantly elevated, which increases your cardiovascular risk.
It is important to work closely with your healthcare provider to manage these factors through lifestyle changes and possibly medication.
Regular monitoring and follow-up visits are recommended to reduce the risk of complications.

User: What lifestyle changes would you recommend first?

Model: I recommend focusing on reducing salt intake, increasing physical activity gradually, and adopting a heart-healthy diet rich in fruits, vegetables, and whole grains.
Managing stress and avoiding smoking are also important.

User: Should medication be started immediately?

Model: Medication may be necessary depending on your healthcare provider's assessment.
It's important to discuss this with your doctor, who will consider your overall risk and current health status before prescribing any treatment.


Patient info: The patient is a 50 years old Male.
BMI is 21.97.
The patient does not smoke, does not drink alcohol, and is physically active.
Blood pressure is 110 / 80.
Blood pressure category: Hypertension Stage 1.
Cholesterol is normal.
Glucose is normal. Does the patient have heart disease? Answer: No

Advice: Your vital signs and lab results are within normal ranges, which is reassuring.
Continue maintaining your healthy lifestyle, including regular physical activity and a balanced diet, to keep your heart in good condition.
Regular check-ups are still important to monitor any future changes.

User: How can I lower my hypertension?

Model:'''

In [ ]:
output3 = pipe(prompt3,
              max_new_tokens = 200,
              #temperature = 0.5, # controls randomness for more varied outputs
              #top_p = 0.95, # controls how much prob mass is considered (top 95%)
              repetition_penalty = 1.2, # penalises repeating tokens
              do_sample = False,
              return_full_text = False, # prompt included
              #early_stopping = True, # stops generating when EOS token is reached
             pad_token_id = tokenizer.eos_token_id)

print(output3[0]["generated_text"])

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 To help control high blood pressure:
• Reduce sodium (salt) consumption
• Increase potassium-rich foods such as bananas or potatoes
• Exercise regularly for at least 30 minutes per day
• Avoid tobacco use
